# Namespaces Lab

In this lab, we will explore how namespaces are presented in the Linux shell and how you can interact with namespaces using some simple commands. During the lab we will explore how you can interact with both the Network and Mount namespaces, which are commonly used by containers as an isolation mechanism to separate workloads running a single host.

To start, lets explore a simple example of how namespaces work. Lets look at the network namespace.

Here are the interfaces on our host:

In [1]:
ip link

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9001 qdisc mq state UP mode DEFAULT group default qlen 1000
    link/ether 0a:49:1b:58:ba:8a brd ff:ff:ff:ff:ff:ff
3: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP mode DEFAULT group default 
    link/ether 02:42:7e:db:91:d2 brd ff:ff:ff:ff:ff:ff
21: veth16fe27a@if20: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP mode DEFAULT group default 
    link/ether 82:c1:9b:ec:83:6c brd ff:ff:ff:ff:ff:ff link-netnsid 1


Now, Lets use unshare to create a new namespace and run the same command to see the interfaces in this namespace:

In [2]:
sudo unshare -n ip link

1: lo: <LOOPBACK> mtu 65536 qdisc noop state DOWN mode DEFAULT group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00


Note, we have a loopback interface, but the parameters attached to this interface are different to loopback on the host OS. This is a separate logical interface which has been facilitated by namespaces.

## Docker namespace utilization demo

Now lets take a look at how docker uses namespaces to separate containers, and explore how we can interact with this namespace from the host OS. 

To start, lets run a redis container:

In [3]:
docker pull redis

Using default tag: latest
latest: Pulling from library/redis

3a3ba0a5: Pulling fs layer 
019a4b43: Pulling fs layer 
73b37fb2: Pulling fs layer 
0dfbb7e3: Pulling fs layer 
f5ba1240: Pulling fs layer 
Digest: sha256:000339fb57e0ddf2d48d72f3341e47a8ca3b1beae9bdcb25a96323095b72a79b
Status: Downloaded newer image for redis:latest


In [22]:
docker run --name redis -d redis

a7464de7a665a2eafadc572b0a27b2663e969d8d983b26ac0e466858d032f4be


For the purposes of this lab, we will need to know the PID the Redis container is running under from the Host OS. We can extract this with the following shell command:

In [23]:
REDIS_PID=$(ps ax | grep redis | grep -v grep | awk '{print $1}')
echo $REDIS_PID

30740


From the Host OS, we can see the redis process.. even though redis is in a separate namespace. This is because the PID namespace is a little bit different and is visible to the host OS to allow us to see all running processes within the kernel. 

Next, we can find the network namespace id for our process by exploring the /proc filesystem with the readlink command:

In [25]:
sudo readlink /proc/$REDIS_PID/ns/net

net:[4026532522]


Lets look deeper and examine the networking configuration of the Redis container we've started up. First, lets pull some information from Docker to gather the IP address recorded for the container: 

In [26]:
CONTAINER_IP=$(docker inspect redis | jq .[0].NetworkSettings.IPAddress | sed -e s/\"//g)
echo $CONTAINER_IP

172.17.0.3


Now to cross check, lets enter the namespace created for the redis-server process (aka the Docker container), and run the ifconfig command. We should see the same IP address as recorded by Docker:

In [27]:
sudo nsenter -t $REDIS_PID -n ifconfig

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.0.3  netmask 255.255.0.0  broadcast 172.17.255.255
        ether 02:42:ac:11:00:03  txqueuelen 0  (Ethernet)
        RX packets 11  bytes 866 (866.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



As docker uses a bridge network by default, I can telnet to the redis-server from the host OS

In [28]:
(printf "PING\r\n";) | nc $CONTAINER_IP 6379 

+PONG


Now lets explore how we can interact with the container namespace from the host OS.

To do this, lets launch a nginx process from the Host OS, in the redis container namespace. 

Now, nginx does not ship in the docker container for redis, and the redis container is based on Ubuntu Linux, but we do have nginx installed on our Host OS, running Amazon Linux. 

In the next example, we namespace for our redis server, but execute nginx from our host OS, which is running Amazon Linux:

In [29]:
sudo nsenter -t $REDIS_PID -n nginx

Now, to validate the nginx process is not running in the host OS network namespace, i can try to connect to localhost:

In [30]:
curl http://localhost

curl: (7) Failed to connect to localhost port 80: Connection refused


: 7

However, if i connect to the IP associated with the redis container network namespace:

In [31]:
curl http://$CONTAINER_IP

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en">
    <head>
        <title>Test Page for the Nginx HTTP Server on Amazon Linux</title>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
        <style type="text/css">
            /*<![CDATA[*/
            body {
                background-color: #fff;
                color: #000;
                font-size: 0.9em;
                font-family: sans-serif,helvetica;
                margin: 0;
                padding: 0;
            }
            :link {
                color: #c00;
            }
            :visited {
                color: #c00;
            }
            a:hover {
                color: #f50;
            }
            h1 {
                text-align: center;
                margin: 0;
                padding: 0.6em 2em 0.4em;
                background-color: #294172;
                colo

You can see this is the Amazon Linux distribution of Redis. 

I'm running a process in the same networking space as my container, but not in the container. It's sharing the networking space, but it's not running under Ubuntu which is what the container is running

Interacting with namespaces on a host can be a useful tool for troubleshooting. By using commands like the above above, we can interact with the networking stack the docker container is running in. In turn, we can use this for troubleshooting to perform tests and rule out configuration problems in the docker container without needing to rebuild the container and redeploy. 

## Exploring the mount namespace

Lets now look at this from the opposite direction: We don't have redis installed on the Amazon Linux host OS, but we do have this as part of our redis container that is running. Lets explore accessing the mount namespace from the Host OS. 

First, lets show there is no redis-server installed or running on localhost:

In [39]:
which redis-server

/usr/bin/which: no redis-server in (/home/ec2-user/.local/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/ec2-user/.local/bin:/home/ec2-user/bin)


: 1

In [40]:
(printf "PING\r\n";) | nc localhost 6379 

Ncat: Connection refused.


: 1

Next, we can enter the mount namespace for our redis container, and run this binary on the host OS:

In [41]:
sudo nsenter -t $REDIS_PID -m /usr/local/bin/redis-server &

[1] 30866


: 1

Now we can test the redis server in the host OS network namespace:

In [42]:
(printf "PING\r\n";) | nc localhost 6379 

+PONG


We can also see we have two instances of redis running:

In [43]:
ps ax | grep redis | grep -v grep

30740 ?        Ssl    0:00 redis-server *:6379
30866 pts/2    S      0:00 sudo nsenter -t 30740 -m /usr/local/bin/redis-server
30867 pts/2    Sl     0:00 /usr/local/bin/redis-server *:6379


Lab Cleanup

After running this demo, use the following commands to clean up the lab and stop any running redis-service processes:

In [8]:
docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
24c69cb086e0        amazonlinux:2       "/bin/tail -f /dev/n…"   15 minutes ago      Up 15 minutes                           blissful_gates


In [7]:
docker stop redis

redis


In [35]:
ps -ef | grep redis

libstor+ 30740 30699  0 02:27 ?        00:00:00 redis-server *:6379
root     30846 30226  0 02:34 pts/2    00:00:00 sudo nsenter -t 30740 -m /usr/local/bin/redis-server
root     30847 30846  0 02:34 pts/2    00:00:00 /usr/local/bin/redis-server *:6379
ec2-user 30854 30226  0 02:34 pts/2    00:00:00 grep --color=auto redis


In [36]:
sudo kill -9 30846 30847

[1]+  Killed                  sudo nsenter -t $REDIS_PID -m /usr/local/bin/redis-server


In [21]:
docker rm redis

redis
